In [ ]:
from sqlalchemy import create_engine
import pandas as pd
# Open database connection
engine = create_engine("mysql+pymysql://tcrd@tcrd.kmc.io:3306/tcrd6134pharos2") # latest version

In [ ]:
## fetch all the required TCRD tables... 7 of them 
dfprotein = pd.read_sql_query("""
  select 
  protein.id,
  protein.uniprot,
  protein.name,
  protein.sym,
  target.tdl
from
  protein
  join t2tc on protein.id=t2tc.protein_id
  join target on t2tc.target_id=target.id
order by
  protein.id
 """, engine)

dfgo = pd.read_sql_query("""
select
  goa.protein_id,
  goa.go_id,
  target.tdl
from
  goa
  join t2tc on goa.protein_id=t2tc.protein_id
  join target on t2tc.target_id=target.id
order by
  protein_id
""", engine)

dfpathway = pd.read_sql_query("""
select
  pathway.protein_id,
  pathway.id_in_source,
  pathway.pwtype
from
pathway
  join t2tc on pathway.protein_id=t2tc.protein_id
  join target on t2tc.target_id=target.id
order by
  protein_id
""", engine)
dfpathway = dfpathway.loc[(dfpathway["pwtype"] == "Reactome") | (dfpathway["pwtype"] == "KEGG")]

dfppi = pd.read_sql_query("""
select
  ncats_ppi.protein_id,
  ncats_ppi.ppitypes,
  ncats_ppi.score,
  ncats_ppi.other_id
from
ncats_ppi
  join t2tc on ncats_ppi.protein_id=t2tc.protein_id
WHERE ppitypes = 'STRINGDB'
order by
  protein_id

""", engine)
dfppi.rename(columns={'protein_id': 'protein_id1', 'other_id':'protein_id2'}, inplace=True)
res = (dfppi[~dfppi.filter(like='protein_id').apply(frozenset, axis=1).duplicated()]
       .reset_index(drop=True))
res2 = res.pivot_table(
    values='score', index='protein_id1', columns='protein_id2')
res2.columns.name = None
dfppi = res2.reset_index()
dfppi = dfppi.add_prefix('PPI:')
dfppi.rename(columns = {'PPI:protein_id1':'protein_id'}, inplace=True)

dfhpa = pd.read_sql_query("""
select
  expression.protein_id,
  expression.tissue,
  expression.etype,
  expression.oid
from
expression
  join t2tc on expression.protein_id=t2tc.protein_id
WHERE etype = 'HPA Protein'
order by
  protein_id
""", engine)
dfhpa['tissue'] = 'HPA:' + dfhpa['tissue'].astype(str)

dfgtex = pd.read_sql_query("""
select
  gtex.protein_id,
  gtex.tissue,
  gtex.tpm,
  gtex.uberon_id
from
gtex
  join t2tc on gtex.protein_id=t2tc.protein_id
order by
  protein_id
""", engine)
dfgtex['tissue'] = 'GTEX:' + dfgtex['tissue'].astype(str)

dfdis = pd.read_sql_query("""
select
 disease.protein_id,
 disease.name,
 disease.mondoid
from
  disease
  join t2tc on disease.protein_id=t2tc.protein_id
order by
  protein_id
""", engine)

# Data for GTEX

In [ ]:
# Combine all the dfs 
dfprotein.rename(columns={'id': 'protein_id'}, inplace=True)

df = dfhpa[dfhpa['tissue'].notna()]
hpadf = pd.crosstab(df["protein_id"], df["tissue"]).reset_index().rename_axis(columns=None)

df = dfgo[dfgo['go_id'].notna()]
godf = pd.crosstab(df["protein_id"], df["go_id"]).reset_index().rename_axis(columns=None)

df = dfpathway[dfpathway['id_in_source'].notna()]
pathwaydf = pd.crosstab(df["protein_id"], df["id_in_source"]).reset_index().rename_axis(columns=None)

df = dfppi
ppidf = df 

df = dfgtex[dfgtex['tissue'].notna()]
gtexdf = pd.crosstab(df["protein_id"], df["tissue"]).reset_index().rename_axis(columns=None)

df1 = dfprotein.set_index('protein_id')
df2 = godf.set_index('protein_id')
df3 = pathwaydf.set_index('protein_id')
df4 = hpadf.set_index('protein_id')
df5 = ppidf.set_index('protein_id')

df6 = gtexdf.set_index('protein_id')
gtex = df6
df = pd.concat([df1,df2,df3,df4, df5, df6],axis=1,sort=False).reset_index()
df.rename(columns = {'index':'protein_id'})
df.to_feather("v3data2020.feather")

# Data for LINCS

In [ ]:
dflincs = pd.read_table('lincs.tsv') # lincs data from pharos at: https://drive.google.com/drive/folders/1P3-cBICdSzdXf7fiVS9qtKjdMZLANnwG?usp=sharing
dflincs = dflincs.drop('Unnamed: 0', axis=1)
dflincs = dflincs.add_prefix('LINCS:')
dflincs.rename(columns = {'LINCS:protein_id':'protein_id'}, inplace=True)
df = dflincs
lincsdf = df

df6 = lincsdf.set_index('protein_id')
lincs = df6
df = pd.concat([df1,df2,df3,df4, df5, df6],axis=1,sort=False).reset_index()
df.rename(columns = {'index':'protein_id'})
df.to_feather("v4data2020.feather")

# Data for DISEASES

In [ ]:
df = dfdis[dfdis['mondoid'].notna()]
disdf = pd.crosstab(df["protein_id"], df["mondoid"]).reset_index().rename_axis(columns=None)

df6 = disdf.set_index('protein_id')
dis = df6
df = pd.concat([df1,df2,df3,df4, df5, df6],axis=1,sort=False).reset_index()
df.rename(columns = {'index':'protein_id'})
df.to_feather("v5data2020.feather")

# Data for CCLE

In [ ]:
dfccle = pd.read_table('ccle.tsv') # ccle data from pharos at: https://drive.google.com/drive/folders/1P3-cBICdSzdXf7fiVS9qtKjdMZLANnwG?usp=sharing
dfccle = dfccle.drop('Unnamed: 0', axis=1)
dfccle.columns = dfccle.columns.str.split('(').str[-1]
dfccle.columns = dfccle.columns.str.replace(')', '')

df = dfccle
ccledf = df

df6 = ccledf.set_index('protein_id')
ccle = df6
df = pd.concat([df1,df2,df3,df4, df5, df6],axis=1,sort=False).reset_index()
df.rename(columns = {'index':'protein_id'})
df.to_feather("v6data2020.feather")

# Data for _ALL models

In [ ]:
df = pd.concat([df1,df2,df3,df4, df5, gtex, lincs, dis, ccle],axis=1,sort=False).reset_index()
df.rename(columns = {'index':'protein_id'})
df.dropna(subset=['uniprot'], inplace=True)
df.to_feather('v13data2020.feather')